In [51]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.multiclass import OneVsRestClassifier

In [52]:
train=pd.read_csv("train_preprocessed.csv")

In [53]:
train.head()

,Unnamed: 0,Text,Tags
0,0,two cultur statist vs machin learn last year r...,['machine-learning']
1,1,forecast demograph census way forecast demogra...,['forecasting']
2,2,bayesian frequentist reason plain english woul...,['bayesian']
3,3,mean p valu valu statist test take statist cou...,"['hypothesis-testing', 't-test', 'p-value', 'i..."
4,4,exampl teach correl mean causat old say correl...,['correlation']


In [54]:
train.drop(["Unnamed: 0"], axis = 1,inplace=True)

In [55]:
train.head()

,Text,Tags
0,two cultur statist vs machin learn last year r...,['machine-learning']
1,forecast demograph census way forecast demogra...,['forecasting']
2,bayesian frequentist reason plain english woul...,['bayesian']
3,mean p valu valu statist test take statist cou...,"['hypothesis-testing', 't-test', 'p-value', 'i..."
4,exampl teach correl mean causat old say correl...,['correlation']


In [56]:
train.isnull().values.any()
train = train.dropna(how='any',axis=0)
train.isnull().values.any()

False

In [57]:
X=train['Text']

In [58]:
import ast
train['Tags'] = train['Tags'].apply(lambda x: ast.literal_eval(x))

In [59]:
Tags=train["Tags"]
Tags

0                                       [machine-learning]
1                                            [forecasting]
2                                               [bayesian]
3        [hypothesis-testing, t-test, p-value, interpre...
4                                            [correlation]
                               ...                        
85080    [logistic, categorical-data, interaction, inte...
85081                                       [linear-model]
85082    [machine-learning, hypothesis-testing, statist...
85083                     [hypothesis-testing, self-study]
85084                                 [hypothesis-testing]
Name: Tags, Length: 73527, dtype: object

In [60]:
multilabel=MultiLabelBinarizer()

In [61]:
y=multilabel.fit_transform(Tags)

In [65]:
multilabel.classes_

array(['anova', 'arima', 'autocorrelation', 'bayesian', 'binomial',
       'bootstrap', 'categorical-data', 'chi-squared', 'classification',
       'clustering', 'conditional-probability', 'confidence-interval',
       'correlation', 'covariance', 'cross-validation', 'data-mining',
       'data-transformation', 'data-visualization', 'dataset',
       'distributions', 'econometrics', 'estimation', 'experiment-design',
       'feature-selection', 'forecasting', 'generalized-linear-model',
       'hypothesis-testing', 'interaction', 'interpretation',
       'least-squares', 'linear-model', 'logistic', 'machine-learning',
       'mathematical-statistics', 'matlab', 'maximum-likelihood', 'mean',
       'mixed-model', 'model-selection', 'modeling',
       'multilevel-analysis', 'multiple-comparisons',
       'multiple-regression', 'multivariate-analysis', 'neural-networks',
       'nonparametric', 'normal-distribution', 'optimization', 'p-value',
       'panel-data', 'pca', 'poisson', 'predi

In [66]:
pd.DataFrame(y,columns=multilabel.classes_)

,anova,arima,autocorrelation,bayesian,binomial,bootstrap,categorical-data,chi-squared,classification,clustering,conditional-probability,confidence-interval,correlation,covariance,cross-validation,data-mining,data-transformation,data-visualization,dataset,distributions,econometrics,estimation,experiment-design,feature-selection,forecasting,generalized-linear-model,hypothesis-testing,interaction,interpretation,least-squares,linear-model,logistic,machine-learning,mathematical-statistics,matlab,maximum-likelihood,mean,mixed-model,model-selection,modeling,multilevel-analysis,multiple-comparisons,multiple-regression,multivariate-analysis,neural-networks,nonparametric,normal-distribution,optimization,p-value,panel-data,pca,poisson,prediction,predictive-models,probability,python,r,random-forest,random-variable,references,regression,repeated-measures,sample-size,sampling,self-study,simulation,spss,standard-deviation,stata,statistical-significance,survival,svm,t-test,time-series,variance
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73522,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
73523,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
73524,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
73525,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0


In [92]:
x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [93]:
tfidf=TfidfVectorizer(ngram_range=(1,2),max_features=100000,max_df=0.8,min_df=3,stop_words="english")
x_train=tfidf.fit_transform(x_train)

In [94]:
x_test=tfidf.transform(x_test)

In [95]:
len(tfidf.vocabulary_)

100000

In [96]:
259058-100000

159058

In [97]:
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import f1_score,precision_score,recall_score
import warnings
warnings.filterwarnings("ignore")

In [98]:
lr=LogisticRegression(solver='saga',C=20)
clf=OneVsRestClassifier(lr)
clf.fit(x_train,y_train)

OneVsRestClassifier(estimator=LogisticRegression(C=20, solver='saga'))

In [99]:
predictions = clf.predict(x_test)
f1 = f1_score(y_test, predictions, average='micro')

In [100]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, predictions,average='micro')

0.3428968499073502

In [101]:
f1

0.5106823356254169

In [102]:
from sklearn.linear_model import SGDClassifier
lr=SGDClassifier(loss = 'hinge', alpha =0.00001)
clf=OneVsRestClassifier(lr)
clf.fit(x_train,y_train)

OneVsRestClassifier(estimator=SGDClassifier(alpha=1e-05))

In [103]:
predictions = clf.predict(x_test)
f1 = f1_score(y_test, predictions, average='micro')
f1

0.5220475012831879

In [104]:
from sklearn.metrics import jaccard_score
jaccard_score(y_test, predictions,average='micro')

0.35322346403990657

In [105]:
for c_value in [1,10,20,30,50]:
  lr=LogisticRegression(solver='saga',C=c_value)
  clf=OneVsRestClassifier(lr)
  clf.fit(x_train,y_train)
  predictions = clf.predict(x_test)
  f1 = f1_score(y_test, predictions, average='micro')
  print(c_value,f1)

1 0.44910983361612683
10 0.5079941353068491
20 0.510602088220413
30 0.51136
50 0.5127145581691036


In [118]:
lr=LogisticRegression(solver='saga',C=1)
clf=OneVsRestClassifier(lr)
clf.fit(x_train,y_train)
predictions = clf.predict(x_test)
f1 = f1_score(y_test, predictions, average='micro')

50 0.44910983361612683


In [120]:
hamming_loss(y_test, predictions)

0.019722562219502245

In [107]:
from sklearn.linear_model import SGDClassifier
cv_err = []
test_err=[]
a_val = [0.000000001,0.00000001,0.0000001,0.000001,0.00001,0.0001,0.001]
for a in a_val:
  clf = SGDClassifier(loss = 'hinge', alpha = a) 
  clf=OneVsRestClassifier(clf, n_jobs = -1)
  clf.fit(x_train,y_train)
  predictions = clf.predict(x_test)
  f1 = f1_score(y_test, predictions, average='micro')
  print(f1)

0.5026646681655424
0.4967702310608468
0.5022863765672743
0.5058572039333048
0.5209313209313209
0.37654835305226914
0.055878439885162105


In [115]:
clf = SGDClassifier(loss = 'hinge', alpha =0.00001) 
clf=OneVsRestClassifier(clf, n_jobs = -1)
clf.fit(x_train,y_train)
predictions = clf.predict(x_test)
f1 = f1_score(y_test, predictions, average='micro')
print(f1)

0.5233170277195434


In [116]:
from sklearn.metrics import hamming_loss

In [117]:
hamming_loss(y_test, predictions)

0.01855387823564078